In [1]:
import openai

embedding = openai.Embedding.create(
    input="Your text goes here", model="text-embedding-ada-002",
    openai_api_key = 
)["data"][0]["embedding"]
len(embedding)

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.